# Luminosity Calculator

In [1]:
# Author: Andrew Louwagie Gordon
# Date Created: 22May2018
# Last Modified: 12Jun2018 (tweaked by Juan Cabanela)

In [2]:
# Import Block
# Import the necessary packages
import ipywidgets as widgets
import bqplot as bq
import numpy as np
import tempNcolor as tc
import expToLaTeX as e2l
from IPython.display import display
import pythreejs as p3j

In [8]:
# Function Definitions Block
def Star_Temp(T):
    '''
    This function calculates the temperature of the star in Kelvin.
    '''
    temp = T * T_Sun
    temp = round(temp, -2) # Round the temperature to the nearest 100 K
    return temp

def L_Ratio(t, r):
    '''
    This function calculates the ratio of luminosities for the star based on temperature and radius.
    '''
    lum = (r ** 2.0) * (t ** 4.0) # Luminosity calculation in L/L_sun
    
    loglum = np.floor(np.log10(lum))  # Compute log of luminosity and round down
    rnd_val = int(1 - loglum)
    
    if (rnd_val>=0):
        format_str = "{0:."+str(int(rnd_val))+"f}"
    else:
        format_str = "{0:.0f}"
    
    return format_str.format(round(lum, rnd_val))

def ConfigStars(r, t):
    '''
    Determines the radii (in solar radii), temperature (in K), and hexcolor of the two stars assuming 
    they are main sequence stars and returns that information.
    '''
    
    # Determine approximate radius in solar radii for both stars.
    radius1= r

    # Determines the approximate temperature of each star.
    temp1 = t
    
    # Use scalar temperature to estimate hexcolor appropriate to each star
    hexcolor1 = tc.rgb2hex(tc.temp2rgb(temp1))[0]
    
    return (radius1, temp1, hexcolor1)

def w(change=None): # w stands for widgets
    '''
    This function continuously updates the widgets that display information.
    '''
    get_l_ratio = L_Ratio(Temp.value, Rad.value)
    L_Ratio_report.value = get_l_ratio
    Luminosity = float(get_l_ratio) * L_Sun
    latex = e2l.exp2LaTeX(Luminosity)
    Luminosity_report.value = r'\({}\)'.format(latex[1])
    t_star = Star_Temp(Temp.value)
    t_star_report.value = str(t_star)
    
def upd(change=None): # upd stands for update
    '''
    This function continuously updates the color and radius (really scale) of the star.
    '''
    global init_r
    
    t_star = Star_Temp(Temp.value)
    counts = tc.temp2rgb(t_star)
    hex_color = tc.rgb2hex(counts)
    star.material.color = hex_color[0]
    scale_dim = Rad.value/init_r
    star.scale = (scale_dim, scale_dim, scale_dim)
    # star.geometry = p3j.SphereGeometry(radius = Rad.value, widthSegments=32, heightSegments=16)



In [9]:
# Variable Definitions Block
L_Sun = 3.8e26 # Solar luminosity in Watts
T_Sun = 5777 # Solar temperature in Kelvin
t_star = T_Sun # Define variable to be updated later
Luminosity = 1 # Define variable to be updated later
get_l_ratio = 1 # Define variable to be updated later
latex = e2l.exp2LaTeX(Luminosity) # Make a list from the number2LaTeX converter being used

In [10]:
# Widgets Definitions Block
# Radius slider in units of R/R_Sun
Rad = widgets.FloatSlider(
    min=0.2, 
    value=1.0,
    max=15, 
    step=0.1, 
    disabled=False, 
    continuous_update=True, 
    orientation='horizontal', 
    readout=True, 
    readout_format='.1f',
)

# Temperature slider in units of T/T_Sun
Temp = widgets.FloatSlider(
    min=0.5,
    value=1.0,
    max=7.0,
    step=0.1,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Widget to report updated temperature in Kelvin
t_star_report = widgets.Label(
    value = str(int(t_star)),
    readout_format='.0f',
    placeholder = 'Type something',
    disabled = True   
)

# Widget to report updated luminosity in L/L_sun
L_Ratio_report = widgets.Label(
    value = str(get_l_ratio),
    placeholder = 'Type something',
    disabled = True   
)

# Widget to report updated luminosity in Watts 
Luminosity_report = widgets.Label(
    value = r'\({}\)'.format(latex[1]),
    placeholder = 'Type something',
    disabled = True   
)

In [11]:
# Calculations Block
t_star = Star_Temp(Temp.value) # Temperature calculation
get_l_ratio = L_Ratio(Temp.value, Rad.value) # Luminosity ratio calculation
Luminosity = float(get_l_ratio) * L_Sun # Convert luminosity to Watts

In [12]:
# Generate the 3D sphere that represents the star
t_star = Star_Temp(Temp.value) # Get the initial temperature for the star
counts = tc.temp2rgb(t_star) # Convert temperature into rgb counts
hex_color = tc.rgb2hex(counts) # Convert the rgb counts into a hexidecimal value that reflects the color of the star
rad_new = Rad.value # Get the radius of the star

# Set scale factor for radius (10 pixels per solar radius)
scale_factor = 1

# Set viewer size
view_width = 600
view_height = 600

# Set initial parameters based on stellar parameters
(radius1, temp1, hexcolor1) = ConfigStars(Rad.value, Temp.value)
r1 = scale_factor*radius1

# Save initial radius to scale all other radii to this
init_r = r1

# set the scale
scale1 = (r1/init_r, r1/init_r, r1/init_r)

# Create sphere using a texture based on public domain STEREO Heliographic map made with 
# combined data from STEREO-Ahead, STEREO-Behind and SDO from Dec. 30, 2011.  Image downloaded from
#    https://stereo.gsfc.nasa.gov/360blog/ 
# and rescaled.
sunTexture = p3j.ImageTexture(imageUri='sun_surface.jpg')
star = p3j.Mesh(geometry=p3j.SphereBufferGeometry(radius=r1, widthSegments=64, heightSegments=32),
                material=p3j.MeshBasicMaterial(color=hex_color[0], map=sunTexture),
                position=[0, 0, 0], scale=scale1)

# Define viewing region size
xmax=1.5*10

# Makes the scene environment, not sure how the background works yet
scene2 = p3j.Scene(children=[star, p3j.AmbientLight(color='white')], background='black')

# Creates the camera so you can see stuff.  Place the cemera just above the x-axis and orient camera so up
# is along y-axis.
starcam = p3j.PerspectiveCamera(position=[2*xmax, 0.1*xmax, 0], up=[0, 1, 0],
                      children=[p3j.DirectionalLight(color='white', 
                                                 position=[xmax, xmax, xmax], 
                                                 intensity=0.5)])
# Makes a controller to use for the 
controller = p3j.OrbitControls(controlling=starcam, enableRotate=True, enableZoom=False)

# creates the object that gets displayed to the screen
renderer2 = p3j.Renderer(camera=starcam, 
                    scene=scene2, 
                    controls=[controller],
                    width=view_width, height=view_height)

# Use the upd function to continuously update the star in the plot
Temp.observe(upd, names=['value'])
Rad.observe(upd, names=['value'])

# Use the w function to continuously update the calculated values in the display widgets on the bottom
Temp.observe(w, names=['value'])
Rad.observe(w, names=['value'])
                                                                                                
# Define the layout for the final widget to make it presentable
box_layout = widgets.Layout(align_items='center', justify_content = 'flex-end', border='none', width='100%')

# Arrange and display all the widgets in a presentable manner
top_box = widgets.VBox([widgets.Label ("Model Star:"), renderer2], layout = box_layout)
rad_slide = widgets.HBox([widgets.Label ('Radius (R/R_Solar):'),Rad])
temp_slide = widgets.HBox([widgets.Label('Temperature (T/T_Solar):'),Temp])
temp_disp = widgets.HBox([widgets.Label('Temperature (K):'), t_star_report])
lratio_disp = widgets.HBox([widgets.Label('Luminosity (L/L_Solar):'), L_Ratio_report])
lum_disp = widgets.HBox([widgets.Label('Luminosity (W):'), Luminosity_report])
lum_disp.children[1].layout.width = '100px' # Make the widget large enough to handle the LaTeX notation 
bottom_left = widgets.VBox([temp_slide, rad_slide])
bottom_right = widgets.VBox([temp_disp, lratio_disp, lum_disp])
bottom = widgets.HBox([bottom_left, bottom_right])
the_box = widgets.VBox([top_box, bottom], layout = box_layout)
the_box